In [31]:
# IFC Parser
# Based on:
# https://thinkmoult.com/using-ifcopenshell-parse-ifc-files-python.html
# https://github.com/johannesmichael/ifc-python/blob/master/modules/ifc_pset_utils.py
# https://www.kaggle.com/code/ponybiam/introduction-to-ifcopenshell-functions/notebook

import ifcopenshell
import ifcHelper 
import pandas as pd
import numpy as np

In [2]:
try:
  ifc_file = ifcopenshell.open('ifc_test_file.ifc')
  print("File loaded successfully")
except:
  print("An exception occurred")



File loaded successfully


In [3]:
print ("File version:")
print (ifc_file.schema)
## Script is written for IFC2X3

File version:
IFC2X3


In [17]:
no_walls = len(ifc_file.by_type("IfcWall"))
no_windows = len(ifc_file.by_type("IfcWindow"))
no_doors = len(ifc_file.by_type("IfcDoor"))
no_flo_segments = len(ifc_file.by_type("IfcFlowSegment"))

print("There are {} walls, {} windows, {} doors and {} flow segments in the file".format(no_walls,no_windows,no_doors,no_flo_segments))

There are 10 walls, 2 windows, 1 doors and 0 flow segments in the file


In [4]:
## Get all walls by_type: IfcWall
##
walls = ifc_file.by_type('IfcWall')

In [22]:
walls[0]

#199=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH2y',#41,'Basic Wall:Z01:331325',$,'Basic Wall:Z01',#153,#195,'331325')

In [24]:
# print "a_wall"s Name attribute
print("The Name of walls[0] is %s"%(walls[0].ObjectType))
# print "a_wall"s Description attribute
print("The Description of walls[0] is %s"%(walls[0].Description))

The Name of walls[0] is Basic Wall:Z01
The Description of walls[0] is None


In [6]:
## Wall elevation area can be calculated using the volume/thickness

In [29]:
walls_types = list()
for i in range(len(walls)):
    
    walls_types.append(walls[i].ObjectType)
    
## Get wall types

In [7]:
walls_volumes = list()
for i in range(len(walls)):
    quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(walls[i])[0])
    walls_volumes.append(quantities['NetVolume'])
    
## Get wall volumes

In [8]:
walls_thickness = list()
for i in range(len(walls)):
    quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(walls[i])[0])
    walls_thickness.append(quantities['Width']/1000)
    
## Get wall thicknesses

In [9]:
walls_areas = list()
for i in range(len(walls)):
    walls_areas.append(walls_volumes[i]/walls_thickness[i])
    
## Get calculated areas

In [10]:
sum(walls_areas)

552.9350204145787

In [34]:
d = {'Type':walls_types,'Area':walls_areas}
d

{'Type': ['Basic Wall:Z01',
  'Basic Wall:Z01',
  'Basic Wall:Z02',
  'Basic Wall:Z02',
  'Basic Wall:Z01',
  'Basic Wall:Z01',
  'Basic Wall:Z01',
  'Basic Wall:Z01',
  'Basic Wall:Z02',
  'Basic Wall:Z02'],
 'Area': [60.1732341748615,
  71.127796309956,
  48.1087212769524,
  50.0613312769524,
  39.290340237810796,
  39.290340237810796,
  55.847062786182995,
  57.769172786183496,
  65.6335106639342,
  65.6335106639342]}

In [38]:
wall_df = pd.DataFrame(d)

In [39]:
wall_df

,Type,Area
0,Basic Wall:Z01,60.173234
1,Basic Wall:Z01,71.127796
2,Basic Wall:Z02,48.108721
3,Basic Wall:Z02,50.061331
4,Basic Wall:Z01,39.290340
5,Basic Wall:Z01,39.290340
6,Basic Wall:Z01,55.847063
7,Basic Wall:Z01,57.769173
8,Basic Wall:Z02,65.633511
9,Basic Wall:Z02,65.633511


In [42]:
df_new = wall_df.groupby('Type')['Area'].sum()

In [43]:
df_new

Type
Basic Wall:Z01    323.497947
Basic Wall:Z02    229.437074
Name: Area, dtype: float64

In [13]:
status = ifcHelper.get_pset_single_value_by_args(walls[0], 'Pset_WallCommon', 'Status')

In [15]:
print (status)

None


In [12]:
ifcHelper.get_property_single_value(ifcHelper.get_related_property_sets(walls[0]))

AttributeError: 'list' object has no attribute 'HasProperties'

In [50]:
ifcHelper.get_related_property_sets(walls[0])

[#334=IfcPropertySet('0XKS9_JrzCLBvqufOYyH2y',#41,'Constraints',$,(#299,#300,#301,#302,#303,#304,#305,#306,#307,#308,#309)),
 #339=IfcPropertySet('0XKS9_JrzCLBvque0YyH2y',#41,'Dimensions',$,(#314,#315,#316)),
 #344=IfcPropertySet('1H02vUCyz6o9nbkhCg_L0Y',#41,'Other',$,(#317,#318,#319,#320,#321)),
 #349=IfcPropertySet('0XKS9_JrzCLBvqueqYyH2y',#41,'Phasing',$,(#310)),
 #354=IfcPropertySet('0XKS9_JrzCLBvqueyYyH2y',#41,'Structural',$,(#311,#312,#313)),
 #509=IfcPropertySet('1zwVZg6$1EsQDQg_kpdpbs',#41,'Pset_ElementShading',$,(#508)),
 #512=IfcPropertySet('34D5K2kEP7cvuyjb34Ync0',#41,'Pset_ProductRequirements',$,(#511)),
 #514=IfcPropertySet('0wABov6VnEzOE$Egx$i0yM',#41,'Pset_QuantityTakeOff',$,(#397)),
 #517=IfcPropertySet('2A3kqFv615GhOd4kwwOrz6',#41,'Pset_ReinforcementBarPitchOfWall',$,(#516)),
 #522=IfcPropertySet('0XKS9_JrzCLBvqxtyYyH2y',#41,'Pset_WallCommon',$,(#397,#519,#520,#521))]

In [47]:
windows = ifc_file.by_type('IfcWindow')

In [48]:
windows

[#1622=IfcWindow('0XKS9_JrzCLBvqv88YyH1n',#41,'M_Stałe:O02:331504',$,'M_Stałe:O02',#3262,#1614,'331504',1830.,915.),
 #1844=IfcWindow('0XKS9_JrzCLBvqv88YyH1s',#41,'M_Stałe:O01:331511',$,'M_Stałe:O01',#3297,#1836,'331511',609.999999999999,406.)]